In [4]:
import random, os
import numpy as np
import pandas as pd
from functools import reduce
import seaborn as sns
from sklearn import datasets
from sklearn import model_selection
from sklearn.feature_selection import VarianceThreshold
from tqdm import tqdm
from tpot import TPOTRegressor, TPOTClassifier

In [2]:
boston = datasets.load_iris()
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    boston.data, boston.target, test_size=0.2, random_state=0)

In [3]:
tpot = TPOTClassifier(generations=10, population_size=50, verbosity=2, n_jobs=8)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

/Users/oleksandrsavsunenko/anaconda3/envs/data/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


Generation 1 - Current best internal CV score: 0.9662424242424242
Generation 2 - Current best internal CV score: 0.9738181818181818
Generation 3 - Current best internal CV score: 0.9738181818181818
Generation 4 - Current best internal CV score: 0.9829090909090908
Generation 5 - Current best internal CV score: 0.9829090909090908
Generation 6 - Current best internal CV score: 0.9829090909090908
Generation 7 - Current best internal CV score: 0.9829090909090908
Generation 8 - Current best internal CV score: 0.9829090909090908
Generation 9 - Current best internal CV score: 0.9829090909090908
Generation 10 - Current best internal CV score: 0.9829090909090908

Best pipeline: GaussianNB(Normalizer(ZeroCount(input_matrix), norm=l2))
1.0


In [5]:
%timeit pd.read_feather('tmp/bulldozers-raw')

72.9 ms ± 5.18 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [2]:
df_raw = pd.read_feather('tmp/bulldozers-raw')

In [7]:
df_raw.to_csv('tmp/test.csv')

In [10]:
%timeit _ = pd.read_csv('tmp/test.csv')

/Users/oleksandrsavsunenko/anaconda3/envs/data/lib/python3.6/site-packages/IPython/core/magic.py:187: DtypeWarning: Columns (11,13,14,20,23,24,25,26,27,28,29,30,31,33,34,35,36,37,39,40,41,42,43,44,45,46,47,48,49,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  call = lambda f, *a, **k: f(*a, **k)
/Users/oleksandrsavsunenko/anaconda3/envs/data/lib/python3.6/site-packages/IPython/core/magics/execution.py:1151: DtypeWarning: Columns (11,13,14,20,23,24,25,26,27,28,29,30,31,33,34,35,36,37,39,40,41,42,43,44,45,46,47,48,49,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  all_runs = timer.repeat(repeat, number)


3.89 s ± 182 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
from fastai.imports import *
from fastai.structured import *
df, y, nas = proc_df(df_raw, 'SalePrice')

/Users/oleksandrsavsunenko/anaconda3/envs/data/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/Users/oleksandrsavsunenko/anaconda3/envs/data/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [11]:
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics.scorer import make_scorer

from fastai.imports import *
from fastai.structured import *

def rmsle(y, y_pred):
    assert len(y) == len(y_pred)
    terms_to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(terms_to_sum) * (1.0/len(y))) ** 0.5

my_custom_scorer = make_scorer(rmsle, greater_is_better=False
                              )
df_raw = pd.read_feather('tmp/bulldozers-raw')
df, y, nas = proc_df(df_raw, 'SalePrice')
df = df.astype(np.float64)
X_train, X_test, y_train, y_test = train_test_split(df, y,
                                                    train_size=0.005)

tpot = TPOTRegressor(generations=5, population_size=25, verbosity=2, scoring=my_custom_scorer)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

/Users/oleksandrsavsunenko/anaconda3/envs/data/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
/Users/oleksandrsavsunenko/anaconda3/envs/data/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


Generation 1 - Current best internal CV score: -0.390543932874581
Generation 2 - Current best internal CV score: -0.390543932874581
Generation 3 - Current best internal CV score: -0.3797842647675842
Generation 4 - Current best internal CV score: -0.3797842647675842
Generation 5 - Current best internal CV score: -0.34027203185698474

Best pipeline: GradientBoostingRegressor(input_matrix, alpha=0.85, learning_rate=0.1, loss=huber, max_depth=10, max_features=0.9500000000000001, min_samples_leaf=2, min_samples_split=10, n_estimators=100, subsample=0.7500000000000001)
-0.32694210401775226
